<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt

ERROR: cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl is not a supported wheel on this platform.


In [1]:
!python --version

Python 3.11.13


In [3]:
#import os
#from dotenv import load_dotenv
#load_dotenv()
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from google.colab import userdata
OPENAI_API_KEY = userdata.get('apikey_rh')

In [ ]:
!pip install langchain_openai

In [6]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [ ]:
model.invoke("Wie funktioniert ein Ottomotor?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Wie funktioniert ein Ottomotor?")

In [9]:
from langchain.prompts import ChatPromptTemplate

template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ralfs Bruder heißt Axl.", question="Wer ist Ralfs Bruder?")

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".\n\nContext: Ralfs Bruder heißt Axl.\n\nQuestion: Wer ist Ralfs Bruder?\n'

In [10]:
chain = prompt | model | parser
chain.invoke({
    "context": "Ralfs Bruder heißt Axl",
    "question": "Wer ist Ralfs Bruder?"
})

'Ralfs Bruder ist Axl.'

In [11]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [12]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "English",
    }
)

"Ralf's brother Axl has two other siblings, so Ralf has a total of three siblings."

Let's read the transcription and display the first few characters to ensure everything works as expected.

In [14]:
with open("interview.txt") as file:
    interview = file.read()

interview[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

In [15]:
try:
    chain.invoke({
        "context": interview,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

In [ ]:
!pip install langchain-community

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("interview.txt")
text_documents = loader.load()
text_documents

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': 'interview.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them. arranging some"),
 Document(metadata={'source': 'interview.txt'}, page_content='arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow,'),
 Document(metadata={'source': 'interview.txt'}, page_content='buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences'),
 Document(metadata={'source': 'interview.txt'}, page_content="intelligences are kind of like the next stage of development. And I don't know where it leads to."),
 Document(metadata={'source': 'interview.txt'}, page_content='where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These')]

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedded_query = embeddings.embed_query("Wer ist Ralfs Bruder?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[0.01599571667611599, -0.019886909052729607, 0.01371423527598381, -0.013169214129447937, -0.027580568566918373, 0.014462053775787354, -0.013625510968267918, -0.0007193002384155989, 0.0008008948643691838, -0.01744065433740616]


In [22]:
sentence1 = embeddings.embed_query("Ralfs Bruder ist Axl")
sentence2 = embeddings.embed_query("Michaelas Bruder ist Paul")

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(np.float64(0.9093964737341363), np.float64(0.843505451391902))

In [ ]:
!pip install docarray

In [27]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Ralfs Bruder heißt Axl",
        "Michaela und Paul sind Geschwister",
        "Dennis mag weiße Autos",
        "Anna Mutter ist Lehrerin",
        "Hektor fährt einen schwarzen Audi",
        "Michaela hat zwei Geschwister",
    ],
    embedding=embeddings,
)

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [28]:
vectorstore1.similarity_search_with_score(query="Wer ist Ralfs Bruder?", k=3)

[(Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  np.float64(0.9127635862951935)),
 (Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
  np.float64(0.8209651848203824)),
 (Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
  np.float64(0.8025362350493098))]

In [29]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Wer ist Ralfs Bruder?")

[Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
 Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
 Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
 Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi')]

In [30]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Welche Farbe hat Hektors Auto?")

{'context': [Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi'),
  Document(metadata={}, page_content='Dennis mag weiße Autos'),
  Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  Document(metadata={}, page_content='Michaela hat zwei Geschwister')],
 'question': 'Welche Farbe hat Hektors Auto?'}

In [31]:
chain = setup | prompt | model | parser
chain.invoke("Welche Farbe hat Hektors Auto?")

'Hektors Auto ist schwarz.'

In [32]:
chain.invoke("Welches Auto fährt Hektor?")

'Hektor fährt einen schwarzen Audi.'

In [33]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [34]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence refers to advanced artificial intelligence systems that are seen as the next stage of development in AI. These systems are thought to possess capabilities that allow them to uncover and solve complex problems or puzzles within the universe. Unlike traditional forms of AI, synthetic intelligences are anticipated to engage in tasks that involve understanding emotions, creativity, and generating art and ideas autonomously.'